# Construction of Route Graphs

**************************************
This notebook walks you through the code in the [route_graph.py](https://gitlab.crowdai.org/SBB/train-schedule-optimisation-challenge-starter-kit/blob/master/utils/route_graph.py) script. It contains code to build directed graphs in the [networkx](https://networkx.github.io/) package from the `routes` in the problem instances. 

This should help you to better understand the `routes` and how to work with them. It may also prove useful in your solving algorithm, such as for finding zero-penalty paths or the like. If you don't like to work with the networkx package, you can copy the logic from the functions `from_node_id` and `to_node_id`, which assign the node id's, to the graph library of your choice.

To run the following code, please ensure that you use Pyhton >= 3.6 and first install the following Python libraries:
- networkx
- matplotlib
**************************************

Import libraries

In [1]:
import json
import time
import networkx as nx
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import os

from functools import partial
from itertools import chain, product, starmap

This function returns "from"-node id for a given `route_section`. The crucial point is that nodes with common `route_alternative_marker`s are identified as the same node in the graph.

In [2]:
def from_node_id(route_path, route_section, index_in_path):
    if "route_alternative_marker_at_entry" in route_section.keys() and \
            route_section["route_alternative_marker_at_entry"] is not None and \
            len(route_section["route_alternative_marker_at_entry"]) > 0:
                return "(" + str(route_section["route_alternative_marker_at_entry"][0]) + ")"
    else:
        if index_in_path == 0:  # can only get here if this node is a very beginning of a route
            return "(" + str(route_section["sequence_number"]) + "_beginning)"
        else:
            return "(" + (str(route_path["route_sections"][index_in_path - 1]["sequence_number"]) + "->" +
                          str(route_section["sequence_number"])) + ")"

This function returns "to"-node id for a given `route_section`

In [3]:
def to_node_id(route_path, route_section, index_in_path):
    if "route_alternative_marker_at_exit" in route_section.keys() and \
            route_section["route_alternative_marker_at_exit"] is not None and \
            len(route_section["route_alternative_marker_at_exit"]) > 0:

                return "(" + str(route_section["route_alternative_marker_at_exit"][0]) + ")"
    else:
        if index_in_path == (len(route_path["route_sections"]) - 1): # meaning this node is a very end of a route
            return "(" + str(route_section["sequence_number"]) + "_end" + ")"
        else:
            return "(" + (str(route_section["sequence_number"]) + "->" +
                          str(route_path["route_sections"][index_in_path + 1]["sequence_number"])) + ")"

In [4]:
def root_to_leaf_paths(G):
    """Yields root-to-leaf paths in a directed acyclic graph.

    `G` must be a directed acyclic graph. If not, the behavior of this
    function is undefined. A "root" in this graph is a node of in-degree
    zero and a "leaf" a node of out-degree zero.

    When invoked, this function iterates over each path from any root to
    any leaf. A path is a list of nodes.

    """
    roots = (v for v, d in G.in_degree() if d == 0)
    leaves = (v for v, d in G.out_degree() if d == 0)
    all_paths = partial(nx.all_simple_paths, G)
    # TODO In Python 3, this would be better as `yield from ...`.
    chaini = chain.from_iterable
    return chaini(starmap(all_paths, product(roots, leaves)))

## Example
We build the route graphs for the two routes in the Challenge sample_instance.

For large graphs, you probably want to deactivate the printint output.


In [5]:
# scenario = "../sample_files/sample_scenario.json"  # adjust path to the sample instance if it is not located there
scenario = "../problem_instances/01_dummy.json"
with open(scenario) as fp:
    scenario = json.load(fp)
    
# start_time = time.time()

resources = pd.DataFrame.from_dict(scenario['resources'])
resources['release_time'] = pd.to_timedelta(resources['release_time'].apply(lambda x: x.split('PT')[-1]))
resources

,following_allowed,id,release_time
0,False,ZUE_A3-A,00:00:10
1,False,ZUE_A3-B,00:00:10
2,False,ZUE_A4-A,00:00:10
3,False,ZUE_A4-B,00:00:10
4,False,ZUE_A5-A,00:00:10
5,False,ZUE_A5-B,00:00:10
6,False,ZUE_A6-A,00:00:10
7,False,ZUE_A6-B,00:00:10
8,False,ZUE_A7-A,00:00:10
9,False,ZUE_A7-B,00:00:10


In [6]:
service_intentions = pd.DataFrame.from_dict(scenario['service_intentions'])
service_intentions

,id,route,section_requirements
0,18823,18823,"[{'sequence_number': 1, 'section_marker': 'ZLO..."
1,18825,18825,"[{'sequence_number': 1, 'section_marker': 'ZLO..."
2,20423,20423,"[{'sequence_number': 1, 'section_marker': 'ZUE..."
3,20425,20425,"[{'sequence_number': 1, 'section_marker': 'ZUE..."


In [7]:
# now build the graph. Nodes are called "previous_FAB -> next_FAB" within lineare abschnittsfolgen and "AK" if
# there is an Abschnittskennzeichen 'AK' on it
route_graphs = dict()
for route in scenario["routes"]:
    
    print(f"\nConstructing route graph for route {route['id']}")
    # set global graph settings
    G = nx.DiGraph(route_id = route["id"], name="Route-Graph for route "+str(route["id"]))

    # add edges with data contained in the preprocessed graph
    for path in route["route_paths"]:
        for (i, route_section) in enumerate(path["route_sections"]):
            route_section['route_path'] = path['id']
            
#             print("Adding Edge from {} to {} with sequence number {}".format(from_node_id(path, route_section, i), to_node_id(path, route_section, i), route_section))
            G.add_edge(from_node_id(path, route_section, i),
                       to_node_id(path, route_section, i),
                       data=route_section)

    route_graphs[route["id"]] = G

# print("Finished building fahrweg-graphen in {} seconds".format(str(time.time() - start_time)))


Constructing route graph for route 18823

Constructing route graph for route 18825

Constructing route graph for route 20423

Constructing route graph for route 20425


You can try to visualize the graph. Plotting directly from networkx is unfortunately not be as easy to unterstand. This is the reason for outputing graphml files which will allow you to visualize the graph in a tool of your choice.

In [8]:
# route_graph = route_graphs[list(route_graphs.keys())[0]]

# from nxpd import draw
# draw(route_graph, show='ipynb')

In [9]:
def add_requirements_for_path(edges, train):
    edges['section_marker'] = edges[edges['section_marker'].notnull()]['section_marker'].apply(lambda x: x[0])
    edges = edges.drop(['starting_point', 'ending_point', 'route_alternative_marker_at_entry', 'route_alternative_marker_at_exit'], axis=1)

    for moment in ['entry_earliest', 'entry_latest', 'exit_earliest', 'exit_latest', 'min_stopping_time']:
        edges[moment] = np.nan

    for requirement in train:
        for moment in ['entry_earliest', 'entry_latest', 'exit_earliest', 'exit_latest', 'min_stopping_time']:
            if moment in requirement.keys():
                edges.loc[edges['section_marker'] == requirement['section_marker'], moment] = requirement[moment]

        for item in ['entry_delay_weight', 'exit_delay_weight', 'connections']:
            try:
                edges.loc[edges['section_marker'] == requirement['section_marker'], item] = requirement[item]
            except:
                pass
         
    return edges

# edges = add_requirements_for_path(edges, train)
# edges

In [10]:
def preprocess_train(edges):
    edges['entry_earliest'] = pd.to_timedelta(edges['entry_earliest'])
    edges['entry_latest'] = pd.to_timedelta(edges['entry_latest'])
    edges['exit_earliest'] = pd.to_timedelta(edges['exit_earliest'])
    edges['exit_latest'] = pd.to_timedelta(edges['exit_latest'])
    edges['minimum_running_time'] = pd.to_timedelta(edges['minimum_running_time'].apply(lambda x: str(x).split('PT')[-1]))
    edges['min_stopping_time'] = pd.to_timedelta(edges['min_stopping_time'].apply(lambda x: str(x).split('PT')[-1]))
    edges['min_stopping_time'] = edges['min_stopping_time'].fillna(0)
    
    return edges

# edges = preprocess_train(edges)
# edges

In [11]:
def calculate_initial_entry_exit(edges):
    edges['entry_time'] = edges['entry_earliest'].iloc[0]

    exit = [edges['entry_time'].values[0]]
    for i in range(0, len(edges.index)):
        exit.append(exit[i] + edges['minimum_running_time'].iloc[i] + edges['min_stopping_time'].iloc[i])
    edges['exit_time'] = exit[1:]
    edges.loc[1:, 'entry_time'] = edges['exit_time'].shift(1)
    
    return edges

# edges = calculate_initial_entry_exit(edges)
# edges

In [12]:
def calculate_possible_train_paths(G, train, route_id, train_id):
    train_paths = []
    
    for nodes in root_to_leaf_paths(G):
        edges = pd.DataFrame()

        for n1, n2 in zip(nodes[:-1], nodes[1:]):
            edge = pd.DataFrame.from_dict(G.get_edge_data(n1,n2))
            edges = edges.append(edge['data'])
        edges = edges.reset_index(drop=True)

        edges = add_requirements_for_path(edges, train)
        edges = preprocess_train(edges)
        edges = calculate_initial_entry_exit(edges)
        edges['route'] = route_id
        edges['train_id'] = train_id
        train_paths.append(edges)
    return train_paths

In [13]:
def gather_used_resources(used_resources, edges):
    for i, row in edges.iterrows():
#         print(row['resource_occupations'])
        for j in row['resource_occupations']:
            if j['resource'] not in used_resources.keys():
                used_resources[j['resource']] = pd.DataFrame()
            used_resources[j['resource']] = used_resources[j['resource']].append(edges.loc[i, ['train_id', 'entry_time', 'exit_time']])
    #         used_resources[j['resource']] = used_resources[j['resource']].reset_index(drop=True)
    return used_resources

def group_trains(used_resources):
    for resource in used_resources.keys():
        used_resources[resource] = used_resources[resource].groupby('train_id').agg({'entry_time': np.min, 'exit_time': np.max})
    return used_resources

def add_release_time(used_resources):
    for key in used_resources.keys():
        used_resources[key]['exit_time'] += resources[resources['id'] == key]['release_time'].values[0]
    return used_resources

In [14]:
# train_id = 111
trains_with_paths = dict()

for i, service_intention in service_intentions.iterrows():
    train = service_intention['section_requirements']
    route_graph = route_graphs[service_intention['route']]
    route_id = service_intention['route']
    train_id = service_intention['id']

    train_paths = calculate_possible_train_paths(route_graph, train, route_id, train_id)
    trains_with_paths[train_id] = train_paths

# chosen_trains = dict()
# for train in trains_with_paths.keys():
#     chosen_trains[train] = trains_with_paths[train][0]

   
# chosen_trains[list(chosen_trains.keys())[0]]

chosen_trains = []
for train in trains_with_paths.keys():
    chosen_trains.append(trains_with_paths[train][0])

   
chosen_trains[0]

,minimum_running_time,penalty,resource_occupations,route_path,section_marker,sequence_number,entry_earliest,entry_latest,exit_earliest,exit_latest,min_stopping_time,entry_delay_weight,connections,exit_delay_weight,entry_time,exit_time,route,train_id
0,00:00:30,None,"[{'resource': 'ZUE_T31-A', 'occupation_directi...",standard,ZLOE_Halt,1.0,06:35:00,NaT,06:37:00,NaT,00:00:24,1.0,NaN,NaN,06:35:00,06:35:54,18823,18823
1,00:00:32,None,"[{'resource': 'ZUE_T42', 'occupation_direction...",standard,ZLOE,5.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:35:54,06:36:26,18823,18823
2,00:00:08,None,"[{'resource': 'ZUE_T62', 'occupation_direction...",standard,ZUET40,10.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:36:26,06:36:34,18823,18823
3,00:00:14,None,"[{'resource': 'ZUE_T72', 'occupation_direction...",standard,ZUEZSW,15.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:36:34,06:36:48,18823,18823
4,00:00:20,None,"[{'resource': 'ZAU_11', 'occupation_direction'...",standard,ZLST,20.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:36:48,06:37:08,18823,18823
5,00:00:20,None,"[{'resource': 'ZAU_21', 'occupation_direction'...",standard,ZAU90,25.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:37:08,06:37:28,18823,18823
6,00:00:11,None,"[{'resource': 'ZWIE_41', 'occupation_direction...",standard,ZAU,30.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:37:28,06:37:39,18823,18823
7,00:00:14,None,"[{'resource': 'ZWIE_1', 'occupation_direction'...",standard,ZAUZBT,35.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:37:39,06:37:53,18823,18823
8,00:00:10,None,"[{'resource': 'ZWIE_1', 'occupation_direction'...",standard,ZWIE_Halt,37.0,NaT,NaT,06:39:00,NaT,00:00:24,1.0,NaN,NaN,06:37:53,06:38:27,18823,18823
9,00:00:44,None,"[{'resource': 'ZWIE_71', 'occupation_direction...",standard,ZWIE,40.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:38:27,06:39:11,18823,18823


In [15]:
used_resources = dict() 

for train in chosen_trains:
    used_resources = gather_used_resources(used_resources, train)
    
used_resources = group_trains(used_resources)
used_resources = add_release_time(used_resources)

def calculate_resources_collisions_in_seconds(used_resources):
    s = 0
    for resource in used_resources.keys():
        used_resources[resource] = used_resources[resource].sort_values(by=['entry_time'])
        used_resources[resource]['collision'] = (used_resources[resource]['exit_time'] - used_resources[resource]['entry_time'].shift(-1)).dt.total_seconds()
        used_resources[resource].loc[used_resources[resource]['collision'] < 0, 'collision'] = 0
        s += used_resources[resource]['collision'].sum()
    return s
       
calculate_resources_collisions_in_seconds(used_resources)

0.0

In [16]:
def calculate_earliest_violation(chosen_trains, event='entry', time='earliest'):
    s = 0
    for train in chosen_trains:
        requirements = train[train[event + '_' + time].notnull()].copy()
        requirements['violation'] = (requirements[event + '_' + time] - requirements[event + '_time']).dt.total_seconds()
        if time == 'earliest':
            requirements.loc[requirements['violation'] < 0, 'violation'] = 0
        else:
            requirements.loc[requirements['violation'] > 0, 'violation'] = 0
            requirements['violation'] = -requirements['violation']
        s += requirements['violation'].sum()
    return s

print(calculate_earliest_violation(chosen_trains, 'entry', 'earliest'))
print(calculate_earliest_violation(chosen_trains, 'exit', 'earliest'))
print(calculate_earliest_violation(chosen_trains, 'entry', 'latest'))
print(calculate_earliest_violation(chosen_trains, 'exit', 'latest'))

0.0
7948.0
0.0
0.0


In [17]:
def calculate_running_time_violation(chosen_trains):
    s = 0
    for train in chosen_trains:
        train['violation'] = (train['minimum_running_time'] - (train['exit_time'] - train['entry_time'])).dt.total_seconds()
        train.loc[train['violation'] < 0, 'violation'] = 0
        s += train['violation'].sum()
    return s

calculate_running_time_violation(chosen_trains)

0.0

In [18]:
timetables = []
for train in chosen_trains:
    timetable = np.append(train['entry_time'].values[0], train['exit_time'].values).astype(np.float64)
#     print(np.append(train['entry_time'].values[0], train['exit_time'].values))
    timetables.append(timetable)
 
print(timetables)
chain =  np.append(chosen_trains[0]['entry_time'].values[0], chosen_trains[0]['exit_time'].values)
chain 

[array([2.3700e+13, 2.3754e+13, 2.3786e+13, 2.3794e+13, 2.3808e+13,
       2.3828e+13, 2.3848e+13, 2.3859e+13, 2.3873e+13, 2.3907e+13,
       2.3951e+13, 2.3971e+13, 2.3991e+13, 2.4025e+13, 2.4072e+13,
       2.4102e+13, 2.4120e+13, 2.4128e+13, 2.4162e+13, 2.4216e+13,
       2.4228e+13, 2.4238e+13, 2.4271e+13, 2.4299e+13, 2.4324e+13,
       2.4358e+13, 2.4403e+13, 2.4412e+13, 2.4425e+13, 2.4450e+13,
       2.4484e+13, 2.4554e+13, 2.4570e+13, 2.4576e+13, 2.4595e+13,
       2.4629e+13, 2.4681e+13, 2.4695e+13, 2.4703e+13, 2.4722e+13,
       2.4736e+13, 2.4758e+13, 2.4792e+13, 2.4837e+13, 2.4889e+13,
       2.4908e+13, 2.4942e+13, 2.5002e+13, 2.5016e+13, 2.5065e+13,
       2.5103e+13, 2.5118e+13, 2.5152e+13, 2.5217e+13, 2.5226e+13,
       2.5268e+13, 2.5303e+13, 2.5327e+13, 2.5361e+13, 2.5415e+13,
       2.5431e+13, 2.5445e+13, 2.5482e+13, 2.5518e+13, 2.5552e+13,
       2.5618e+13, 2.5651e+13, 2.5666e+13, 2.5700e+13, 2.5773e+13,
       2.5802e+13, 2.5836e+13, 2.5919e+13, 2.5958e+13, 2.5997

array([23700000000000, 23754000000000, 23786000000000, 23794000000000,
       23808000000000, 23828000000000, 23848000000000, 23859000000000,
       23873000000000, 23907000000000, 23951000000000, 23971000000000,
       23991000000000, 24025000000000, 24072000000000, 24102000000000,
       24120000000000, 24128000000000, 24162000000000, 24216000000000,
       24228000000000, 24238000000000, 24271000000000, 24299000000000,
       24324000000000, 24358000000000, 24403000000000, 24412000000000,
       24425000000000, 24450000000000, 24484000000000, 24554000000000,
       24570000000000, 24576000000000, 24595000000000, 24629000000000,
       24681000000000, 24695000000000, 24703000000000, 24722000000000,
       24736000000000, 24758000000000, 24792000000000, 24837000000000,
       24889000000000, 24908000000000, 24942000000000, 25002000000000,
       25016000000000, 25065000000000, 25103000000000, 25118000000000,
       25152000000000, 25217000000000, 25226000000000, 25268000000000,
      

In [24]:
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from itertools import repeat
from collections import Sequence
import math
import time

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -0.5))
creator.create("Individual", numpy.ndarray, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

def give_chain():
#     return chain.astype(np.float64) + 1e11*np.random.randn(len(chain)).astype(np.float64)
    return timetables


def evalOneMax(individual):
    time1 = time.time()
    fitness = []
    for i in individual:
#         used_resources = dict() 
#         for train in range(len(i)):
#             used_resources = gather_used_resources(used_resources, chosen_trains[train])
#         used_resources = group_trains(used_resources)
#         used_resources = add_release_time(used_resources)
        for train in range(len(i)):
            chosen_trains[train]['entry_time'] = i[train][:-1].astype(np.timedelta64)
            chosen_trains[train]['exit_time'] = i[train][1:].astype(np.timedelta64)

            time1 = time.time()
            
            m = 0
            d = 0
                       
#             m += calculate_resources_collisions_in_seconds(used_resources)           
            m += calculate_earliest_violation(chosen_trains, 'entry', 'earliest')
            m += calculate_earliest_violation(chosen_trains, 'exit', 'earliest')
            d += calculate_earliest_violation(chosen_trains, 'entry', 'latest')
            d += calculate_earliest_violation(chosen_trains, 'exit', 'latest')
            m += calculate_running_time_violation(chosen_trains)
            time2 = time.time()
#             print('{:s} function took {:.3f} ms'.format('collisions', (time2-time1)*1000.0))

        fitness.append(m+d)
    time2 = time.time()
    
    return fitness

toolbox.register("attr_bool", give_chain)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def cxTwoPointCopy(ind1, ind2):
#     """Execute a two points crossover with copy on the input individuals. The
#     copy is required because the slicing in numpy returns a view of the data,
#     which leads to a self overwritting in the swap operation. It prevents
#     ::
    
#         >>> import numpy
#         >>> a = numpy.array((1,2,3,4))
#         >>> b = numpy.array((5.6.7.8))
#         >>> a[1:3], b[1:3] = b[1:3], a[1:3]
#         >>> print(a)
#         [1 6 7 4]
#         >>> print(b)
#         [5 6 7 8]
#     """
# #     time1 = time.time()
#     for i in range(len(ind1)):
#         size = len(ind1[0][i])
#         cxpoint1 = random.randint(1, size)
#         cxpoint2 = random.randint(1, size - 1)
#         if cxpoint2 >= cxpoint1:
#             cxpoint2 += 1
#         else: # Swap the two cx points
#             cxpoint1, cxpoint2 = cxpoint2, cxpoint1

#         ind1[0][i][cxpoint1:cxpoint2], ind2[0][i][cxpoint1:cxpoint2] \
#             = ind2[0][i][cxpoint1:cxpoint2].copy(), ind1[0][i][cxpoint1:cxpoint2].copy()
# #     time2 = time.time()
# #     print('{:s} function took {:.3f} ms'.format('cxTwoPointCopy', (time2-time1)*1000.0))
    return ind1, ind2

def mutGaussian(individual, mu, sigma, indpb, shiftpb):
#     time1 = time.time()
    """This function applies a gaussian mutation of mean *mu* and standard
    deviation *sigma* on the input individual. This mutation expects a
    :term:`sequence` individual composed of real valued attributes.
    The *indpb* argument is the probability of each attribute to be mutated.
    :param individual: Individual to be mutated.
    :param mu: Mean or :term:`python:sequence` of means for the
               gaussian addition mutation.
    :param sigma: Standard deviation or :term:`python:sequence` of
                  standard deviations for the gaussian addition mutation.
    :param indpb: Independent probability for each attribute to be mutated.
    :returns: A tuple of one individual.
    This function uses the :func:`~random.random` and :func:`~random.gauss`
    functions from the python base :mod:`random` module.
    """
    size = len(individual)
#     print(individual)
    if not isinstance(mu, Sequence):
        mu = repeat(mu, size)
    elif len(mu) < size:
        raise IndexError("mu must be at least the size of individual: %d < %d" % (len(mu), size))
    if not isinstance(sigma, Sequence):
        sigma = repeat(sigma, size)
    elif len(sigma) < size:
        raise IndexError("sigma must be at least the size of individual: %d < %d" % (len(sigma), size))

    if random.random() < shiftpb:
        for i, m, s in zip(range(size), mu, sigma):      
            individual[i] += random.gauss(m, s)
    for i, m, s in zip(range(size), mu, sigma):      
        if random.random() < indpb:
            individual[i] += random.gauss(m, s)
#     time2 = time.time()
#     print('{:s} function took {:.3f} ms'.format('mutGaussian', (time2-time1)*1000.0))
    return individual,
    
import multiprocessing

pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", cxTwoPointCopy)
toolbox.register("mutate", mutGaussian, mu=0, sigma=1e11, indpb=0.01, shiftpb=0.4)
toolbox.register("select", tools.selTournament, tournsize=3)

/home/max/anaconda3/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/max/anaconda3/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [25]:
%%prun -s cumulative -q -l 100 -T prun0

def main():
    random.seed(64)
    
    pop = toolbox.population(n=40)
    
    # Numpy equality function (operators.eq) between two arrays returns the
    # equality element wise, which raises an exception in the if similar()
    # check of the hall of fame. Using a different equality function like
    # numpy.array_equal or numpy.allclose solve this issue.
    hof = tools.HallOfFame(1, similar=numpy.array_equal)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
#     stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, cxpb=0.4, mutpb=0.2, ngen=100, stats=stats,
                        halloffame=hof)

    return pop, stats, hof

if __name__ == "__main__":
    pop, stats, hof = main()

gen	nevals	avg    	std	min    
0  	40    	14539.2	0  	14539.2
1  	14    	14771.7	1379.2	14310  
2  	19    	14497  	110.331	13995.6
3  	19    	14450.5	1409.73	7354.47
4  	15    	13685  	1938.55	7354.47
5  	23    	13112.5	3204.69	7354.47
6  	24    	9685.38	2952.99	7354.47
7  	15    	7597.04	1278.73	4313.5 
8  	16    	7127.05	801.219	4313.5 
9  	18    	6740.67	1217.87	4246.31
10 	16    	5917.52	1704.09	4246.31
11 	22    	4761.5 	1374.15	1550.38
12 	20    	4046.14	842.227	1550.38
13 	10    	3673.13	1592.65	1499.81
14 	21    	2569.5 	1162.92	1493.35
15 	22    	1808.08	670.496	1450.58
16 	25    	1561.81	335.709	1424.81
17 	25    	1494.96	93.1891	1406.25
18 	18    	1473.39	64.0687	1406.25
19 	14    	1479.91	176.547	1406.25
20 	16    	1430.07	42.8682	1406.25
21 	18    	1441.12	127.958	1406.25
22 	17    	1425.02	81.3439	1406.25
23 	25    	1446.39	152.178	1406.25
24 	17    	1422.59	80.4557	1406.25
25 	26    	1439.28	176.22 	1406.25
26 	25    	1442.97	119.744	1406.25
27 	12    	1433.43	169.228	14

In [26]:
a = hof[0][0]

In [27]:
a

Individual([array([2.42137607e+13, 2.42677607e+13, 2.42997607e+13, 2.43077607e+13,
       2.43217607e+13, 2.43417607e+13, 2.43617607e+13, 2.43727607e+13,
       2.43867607e+13, 2.44207607e+13, 2.44647607e+13, 2.44847607e+13,
       2.45047607e+13, 2.45387607e+13, 2.45857607e+13, 2.46157607e+13,
       2.46337607e+13, 2.46417607e+13, 2.46757607e+13, 2.47297607e+13,
       2.47417607e+13, 2.47517607e+13, 2.47847607e+13, 2.48127607e+13,
       2.48377607e+13, 2.48717607e+13, 2.49167607e+13, 2.49257607e+13,
       2.49387607e+13, 2.49637607e+13, 2.49977607e+13, 2.50677607e+13,
       2.50837607e+13, 2.50897607e+13, 2.51087607e+13, 2.51427607e+13,
       2.51947607e+13, 2.52087607e+13, 2.52167607e+13, 2.52357607e+13,
       2.52497607e+13, 2.52717607e+13, 2.53057607e+13, 2.53507607e+13,
       2.54027607e+13, 2.54217607e+13, 2.54557607e+13, 2.55157607e+13,
       2.55297607e+13, 2.55787607e+13, 2.56167607e+13, 2.56317607e+13,
       2.56657607e+13, 2.57307607e+13, 2.57397607e+13, 2.57817607

In [28]:
chosen_trains[0]['entry_time'] = a[0][:-1].astype(np.timedelta64)
chosen_trains[0]['exit_time'] = a[0][1:].astype(np.timedelta64)
chosen_trains[0]

,minimum_running_time,penalty,resource_occupations,route_path,section_marker,sequence_number,entry_earliest,entry_latest,exit_earliest,exit_latest,min_stopping_time,entry_delay_weight,connections,exit_delay_weight,entry_time,exit_time,route,train_id,violation
0,00:00:30,None,"[{'resource': 'ZUE_T31-A', 'occupation_directi...",standard,ZLOE_Halt,1.0,06:35:00,NaT,06:37:00,NaT,00:00:24,1.0,NaN,NaN,06:43:33.760697,06:44:27.760697,18823,18823,0.0
1,00:00:32,None,"[{'resource': 'ZUE_T42', 'occupation_direction...",standard,ZLOE,5.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:44:27.760697,06:44:59.760697,18823,18823,0.0
2,00:00:08,None,"[{'resource': 'ZUE_T62', 'occupation_direction...",standard,ZUET40,10.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:44:59.760697,06:45:07.760697,18823,18823,0.0
3,00:00:14,None,"[{'resource': 'ZUE_T72', 'occupation_direction...",standard,ZUEZSW,15.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:45:07.760697,06:45:21.760697,18823,18823,0.0
4,00:00:20,None,"[{'resource': 'ZAU_11', 'occupation_direction'...",standard,ZLST,20.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:45:21.760697,06:45:41.760697,18823,18823,0.0
5,00:00:20,None,"[{'resource': 'ZAU_21', 'occupation_direction'...",standard,ZAU90,25.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:45:41.760697,06:46:01.760697,18823,18823,0.0
6,00:00:11,None,"[{'resource': 'ZWIE_41', 'occupation_direction...",standard,ZAU,30.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:46:01.760697,06:46:12.760697,18823,18823,0.0
7,00:00:14,None,"[{'resource': 'ZWIE_1', 'occupation_direction'...",standard,ZAUZBT,35.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:46:12.760697,06:46:26.760697,18823,18823,0.0
8,00:00:10,None,"[{'resource': 'ZWIE_1', 'occupation_direction'...",standard,ZWIE_Halt,37.0,NaT,NaT,06:39:00,NaT,00:00:24,1.0,NaN,NaN,06:46:26.760697,06:47:00.760697,18823,18823,0.0
9,00:00:44,None,"[{'resource': 'ZWIE_71', 'occupation_direction...",standard,ZWIE,40.0,NaT,NaT,NaT,NaT,00:00:00,NaN,NaN,NaN,06:47:00.760697,06:47:44.760697,18823,18823,0.0


In [ ]:
# train['violation'] = (train['minimum_running_time'] - train['exit_time'] - train['entry_time']).dt.total_seconds()

# 8:30 8:32 
# entry exit
# exit - entry = 2

# 4
# minimum
# minimum - (exit - entry)

In [ ]:
# %load_ext line_profiler

In [ ]:
# %lprun -f main()

In [ ]:
print(open('prun0', 'r').read())